This is my first try with CANOES (http://www.columbia.edu/~ys2411/canoes/). From the get-go, we'll use the trimmed samples. Its first step takes a LONG time, so let's do it per sample, and then concatenate it. I tested it before with two samples, and it worked fine.

In [ ]:
%%bash

cd ~/data/cnv/trimmed/canoes
cp ../conifer/samples.txt .
probes="/data/NCR_SBRB/simplex/SeqCapEZ_Exome_v3.0_Design_Annotation_files/SeqCap_EZ_Exome_v3_hg19_capture_targets.bed"

while read s; do
    echo "cd ~/data/cnv/trimmed/canoes/; bedtools multicov -bams ../BAM/${s}.bam -bed $probes -q 20 > ${s}.reads.txt" >> swarm.reads;
done < samples.txt
swarm --time 2-00:00:00 -f swarm.reads --job-name canoes --logdir trash --module bedtools

Then, to combine them:

In [ ]:
# terminal
module load GATK
exome_targets="/data/NCR_SBRB/simplex/SeqCapEZ_Exome_v3.0_Design_Annotation_files/SeqCap_EZ_Exome_v3_hg19_capture_targets.bed"
ref_fa='/fdb/igenomes/Homo_sapiens/UCSC/hg19/Sequence/WholeGenomeFasta/genome.fa'
java -Xmx2000m -Djava.io.tmpdir=/lscratch/${SLURM_JOBID} -jar $GATK_JAR \
    -T GCContentByInterval -L ${exome_targets} -R $ref_fa -o gc.txt
    
for f in `ls CLIA*reads.txt`; do cut -f 5 ${f} > ${f}.cut; done
# check that they all ran fine
wc -l *cut

paste *cut > samples.reads.txt.cut
cut -f 1,2,3 CLIA_400142.reads.txt > first_columns.txt
paste first_columns.txt samples.reads.txt.cut > canoes.reads.txt
ls -1 C*cut | cut -d"." -f 1 > sample_names.txt

Then, in R:

In [ ]:
# but really, in terminal

gc <- read.table("gc.txt")$V2
canoes.reads <- read.table("canoes.reads.txt")
sample.names = read.table("sample_names.txt")
sample.names = sapply(sample.names, as.character)
names(canoes.reads) <- c("chromosome", "start", "end", sample.names)

target <- seq(1, nrow(canoes.reads))
canoes.reads <- cbind(target, gc, canoes.reads)

source('/data/NCR_SBRB/software/CANOES.R')
xcnv.list <- vector('list', length(sample.names))
for (i in 1:length(sample.names)){
    print(i)
    xcnv.list[[i]] <- CallCNVs(sample.names[i], canoes.reads)
}


We can also try to parallelize this:

In [ ]:
library(parallel)
cl = makeCluster(12, type='FORK')
# clusterEvalQ(cl, source('/data/NCR_SBRB/software/CANOES.R'))
xcnv.list = parLapply(cl, sample.names, CallCNVs, canoes.reads)
stopCluster(cl)

xcnvs <- do.call('rbind', xcnv.list)
write.table(xcnvs, file='~/data/cnv/trimmed/canoes/cnvs.txt', row.names=F)

The webpage (http://www.columbia.edu/~ys2411/canoes/) also shows some examples of how to do genotyping for each sample, but I don't think that's necessary? Maybe...

# junk

Well, that's disheartening... do we have anyone ready?

In [3]:
%%bash

cd ~/data/cnv/mdts

echo "Samples ready:"
while read s; do
    if [ -e /data/NCR_SBRB/ADHDQTL/GATK/BAM/${s}/${s}_trimmed_sorted_RG_markduplicate_recalibrated.bam ]; then
        echo $s;
    fi;
done < samples.txt

Samples ready:


Alright then... we'll need to re-run everyone.

Since we're re-running this, it'll be interesting to try different BAM pipelines:

1. Standard (BWA MEM + Picard + Base recalibration)
2. No base recalibration (BWA MEM + Picard)
3. mrsFast (as suggested in CONIFER)

Running mrsFast might not be worth it, and it's somewhat older software (2014). So, if we end up doing it, we might as well run some of their own CNV tools (http://mrcanavar.sourceforge.net/manual.html)

In [ ]:
%%bash

cd /data/NCR_SBRB/simplex

while read s; do
    echo "bash ~/autodenovo/gatk_upToSingleCalls.sh $s" >> swarm.single;
done < ~/data/cnv/mdts/samples.txt;

[sudregp@cn2350 simplex]$ head swarm.single
bash ~/autodenovo/gatk_upToSingleCalls.sh CLIA_400185
bash ~/autodenovo/gatk_upToSingleCalls.sh CLIA_400200
bash ~/autodenovo/gatk_upToSingleCalls.sh CLIA_400193
bash ~/autodenovo/gatk_upToSingleCalls.sh CLIA_400195
bash ~/autodenovo/gatk_upToSingleCalls.sh CLIA_400148
bash ~/autodenovo/gatk_upToSingleCalls.sh CLIA_400188
bash ~/autodenovo/gatk_upToSingleCalls.sh CLIA_400189
bash ~/autodenovo/gatk_upToSingleCalls.sh CLIA_400191
bash ~/autodenovo/gatk_upToSingleCalls.sh CLIA_400181
bash ~/autodenovo/gatk_upToSingleCalls.sh CLIA_400203

[sudregp@cn2350 simplex]$ wc -l swarm.single
84 swarm.single
[sudregp@cn2350 simplex]$ swarm -f swarm.single -t 32 -g 120 --job-name gatk_single --logdir trash --time=48:00:00 --gres=lscratch:1
00
63296498

In [ ]:
%%R

library(MDTS, lib='~/R')
library(BSgenome.Hsapiens.UCSC.hg19)
genome = BSgenome.Hsapiens.UCSC.hg19
map_file = "chr1.map.bw"
setwd('~/data/cnv/mdts')
pD = getMetaData("simplex84.ped")
bins = calcBins(pD, n=5, readLength=100, medianCoverage=150,
                minimumCoverage=5, genome=genome, mappabilityFile=map_file)

# it might be possible to make this go faster if we split the genome by chromosomes?

counts = calcCounts(pD, bins, rl=100)
mCounts <- normalizeCounts(counts, bins)
md <- calcMD(mCounts, pD)
cbs <- segmentMD(md=md, bins=bins)
denovo <- denovoDeletions(cbs, mCounts, bins)

NEED TO DOWNLOAD MAPABILITY FILES FOR ALL CHROMOSOMES. SOMETHING LIKE THIS: http://hgdownload.soe.ucsc.edu/goldenPath/hg19/encodeDCC/wgEncodeMapability/. BUT SHOULD HECK THE QC FILES FOR A BETTER READ LENGTH ESTIMATE.

# junk

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
%matplotlib inline

In [4]:
import glob
ped_files = ['/data/sudregp/multiplex_simplex/simplex.ped']
wes_prefix = ['CLIA', 'CCGO', 'WPS']
# fid = open('/home/sudregp/data/multiplex_simplex/samples_simplex_all.txt', 'r')
# exclude_list = [line.rstrip() for line in fid]
# fid.close()

# no controls/affected pair for comparison
exclude_list = ['CLIA_400165', 'CLIA_400164', 'CLIA_400155', 'CLIA_400146',
                'CLIA_400145', 'CLIA_400126', 'CLIA_400207', 'CLIA_400208',
                'CLIA_400209']
# missing one parent
exclude_list += ['CLIA_400169', 'CLIA_400168']
# family 9030
exclude_list += ['CCGO_800978', 'CCGO_800977', 'CCGO_800976', 'CCGO_800979',
                 'CCGO_800980', 'CLIA_400067']

trios = []
affected = []
controls = []
samples = []
famids = []
sexes = {}
for ped_file in ped_files:
    fid = open(ped_file, 'r')
    for line in fid:
        famid, sid, fa, mo, sex, aff = line.rstrip().split('\t')
        # if the current ID and its parents have WES data, and the sample is 
        # not in yet
        if (fa.split('_')[0] in wes_prefix and
            mo.split('_')[0] in wes_prefix and
            sid.split('_')[0] in wes_prefix and
            sid not in samples and
            (sid not in exclude_list or fa not in exclude_list or mo not in exclude_list)):
            fam = {}
            fam['child'] = sid
            if aff == '1':
                affected.append(sid)
            else:
                controls.append(sid)
            fam['father'] = fa
            fam['mother'] = mo
            fam['famid'] = famid
            sexes[sid] = sex
            trios.append(fam)
            samples += [sid, fa, mo]
            famids.append(famid)
    fid.close()
samples = set(samples)
famids = set(famids)
kids = set(affected + controls)
good_kids = kids

print 'Unique samples:', len(samples)
print 'Unique families:', len(famids)
print 'Unique children:', len(kids)

Unique samples: 84
Unique families: 19
Unique children: 46


In [6]:
fid = open('/data/sudregp/cnv/mdts/simplex84.ped', 'w')
fid.write('subj_id\tfamily_id\tfather_id\tmother_id\tgender\tbam_path\n')
for trio in trios:
    fid.write('\t'.join([trio['child'], trio['famid'], trio['father'],
                         trio['mother'], sexes[trio['child']],
                         'BAM/%s.bam' % trio['child']]) + '\n')
    fid.write('\t'.join([trio['father'], trio['famid'], '0', '0', '1',
                         'BAM/%s.bam' % trio['father']]) + '\n')
    fid.write('\t'.join([trio['mother'], trio['famid'], '0', '0', '2',
                         'BAM/%s.bam' % trio['mother']]) + '\n')
fid.close()

I'm running it in a R terminal to have a better look at the output, but here's a history of the commands (following from https://github.com/JMF47/MDTS/blob/master/vignettes/mdts.Rmd):

Can't tell yet if the chr1 file is enough, or if they need one per each chromosome. Also, need to check if this is the best reference file for us, and check if the other parameters are also appropriate.

Based on the output I'm seeing so far, I'd need one map_file for each chromossome. For example, the output of calcBins looks like this so far:

# TODO

*